In [1]:
"""
Product Matcher v4 - GPU Optimized (RTX 3050 Ready)
===================================================
- Fixed deprecated parameters
- Optimized for 4GB VRAM
- FAISS runs on CPU (GPU version optional)
- Encoder + Reranker run on GPU
"""

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
import re
from tqdm import tqdm
from typing import List, Dict, Tuple, Optional
import warnings
import gc
import os
from pathlib import Path
import torch

warnings.filterwarnings('ignore')


def check_gpu_status():
    """Checks and prints GPU status."""
    print("=" * 60)
    print("🔍 GPU CHECK")
    print("=" * 60)
    
    print(f"\n📦 PyTorch version: {torch.__version__}")
    print(f"🔧 CUDA available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print(f"🎮 CUDA version: {torch.version.cuda}")
        print(f"📊 GPU count: {torch.cuda.device_count()}")
        
        for i in range(torch.cuda.device_count()):
            props = torch.cuda.get_device_properties(i)
            print(f"\n   GPU {i}: {props.name}")
            print(f"   Memory: {props.total_memory / 1e9:.1f} GB")
            print(f"   Compute capability: {props.major}.{props.minor}")
        
        print(f"\n💾 Current VRAM usage:")
        print(f"   Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"   Cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    else:
        print("⚠️  GPU unavailable! Using CPU.")
    
    print(f"\n📦 FAISS GPU support: {hasattr(faiss, 'StandardGpuResources')}")
    print("=" * 60)
    
    return torch.cuda.is_available()


class ProductMatcherGPU:
    """
    GPU-optimized product matcher:
    1. Retrieval: BGE-M3 (GPU) + FAISS -> Top-K candidates
    2. Reranking: BGE-reranker-v2-m3 (GPU) -> Best candidate
    """
    
    _encoder_instance = None
    _reranker_instance = None
    
    def __init__(self, 
                 encoder_model: str = 'BAAI/bge-m3',
                 reranker_model: str = 'BAAI/bge-reranker-v2-m3',
                 cache_dir: str = './cache',
                 use_fp16: bool = True,
                 gpu_id: int = 0,
                 force_gpu: bool = False):
        """
        Args:
            encoder_model: Model for embeddings
            reranker_model: Model for reranking
            cache_dir: Directory for cache storage
            use_fp16: Use float16 to save VRAM
            gpu_id: GPU ID to use
            force_gpu: Raise error if GPU is not found
        """
        
        self.gpu_available = torch.cuda.is_available()
        self.gpu_id = gpu_id
        
        if force_gpu and not self.gpu_available:
            raise RuntimeError("❌ GPU not found!")
        
        # Initialize attributes
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        self.faiss_index: Optional[faiss.Index] = None
        self.gpu_resources = None
        self.eva_texts_clean: Optional[List[str]] = None
        self.eva_df: Optional[pd.DataFrame] = None
        
        # Device setup
        if self.gpu_available:
            self.device = f'cuda:{gpu_id}'
            torch.cuda.set_device(gpu_id)
        else:
            self.device = 'cpu'
        
        print("=" * 60)
        print("🚀 Initializing ProductMatcherGPU")
        print("=" * 60)
        print(f"🖥️  Device: {self.device}")
        
        if self.gpu_available:
            props = torch.cuda.get_device_properties(gpu_id)
            self.vram_total = props.total_memory / 1e9
            print(f"🎮 GPU: {props.name}")
            print(f"💾 VRAM: {self.vram_total:.1f} GB")
            
            # FAISS-GPU (Optional)
            self.faiss_gpu_available = hasattr(faiss, 'StandardGpuResources')
            if self.faiss_gpu_available:
                try:
                    self.gpu_resources = faiss.StandardGpuResources()
                    self.gpu_resources.setTempMemory(256 * 1024 * 1024)  # 256MB
                    print(f"🔧 FAISS-GPU: ✅ Available")
                except:
                    self.faiss_gpu_available = False
                    print(f"🔧 FAISS-GPU: ❌ Initialization error")
            else:
                print(f"🔧 FAISS-GPU: ❌ Not installed (Using CPU)")
        else:
            self.vram_total = 0
            self.faiss_gpu_available = False
        
        # Dtype
        if use_fp16 and self.gpu_available:
            self.model_dtype = torch.float16
            print("⚡ Precision: FP16")
        else:
            self.model_dtype = torch.float32
            print("📊 Precision: FP32")
        
        # Load models
        self._load_encoder(encoder_model)
        self._load_reranker(reranker_model)
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        print("\n✅ Initialization complete!")
        print("=" * 60)
    
    def _load_encoder(self, model_name: str):
        """Load encoder onto GPU."""
        if ProductMatcherGPU._encoder_instance is None:
            print(f"\n📥 Loading encoder: {model_name}")
            
            try:
                # Modern method with dtype
                ProductMatcherGPU._encoder_instance = SentenceTransformer(
                    model_name,
                    device=self.device,
                    model_kwargs={
                        'torch_dtype': self.model_dtype,
                    }
                )
            except Exception as e1:
                print(f"   ⚠️ Attempt 1 failed: {e1}")
                try:
                    # Fallback without kwargs
                    ProductMatcherGPU._encoder_instance = SentenceTransformer(
                        model_name,
                        device=self.device
                    )
                    if self.model_dtype == torch.float16 and self.gpu_available:
                        ProductMatcherGPU._encoder_instance.half()
                except Exception as e2:
                    print(f"   ❌ Load error: {e2}")
                    raise
            
            if self.gpu_available:
                try:
                    dev = next(ProductMatcherGPU._encoder_instance.parameters()).device
                    print(f"   ✅ Encoder on: {dev}")
                except:
                    pass
        else:
            print("✅ Encoder already loaded")
        
        self.encoder = ProductMatcherGPU._encoder_instance
    
    def _load_reranker(self, model_name: str):
        """Load reranker onto GPU."""
        if ProductMatcherGPU._reranker_instance is None:
            print(f"\n📥 Loading reranker: {model_name}")
            
            try:
                # Modern method with model_kwargs
                ProductMatcherGPU._reranker_instance = CrossEncoder(
                    model_name,
                    max_length=512,
                    device=self.device,
                    model_kwargs={'torch_dtype': self.model_dtype}
                )
            except Exception as e1:
                print(f"   ⚠️ Attempt 1 failed: {e1}")
                try:
                    # Fallback
                    ProductMatcherGPU._reranker_instance = CrossEncoder(
                        model_name,
                        max_length=512,
                        device=self.device
                    )
                    if self.model_dtype == torch.float16 and self.gpu_available:
                        ProductMatcherGPU._reranker_instance.model.half()
                except Exception as e2:
                    print(f"   ❌ Load error: {e2}")
                    raise
            
            if self.gpu_available:
                try:
                    dev = next(ProductMatcherGPU._reranker_instance.model.parameters()).device
                    print(f"   ✅ Reranker on: {dev}")
                except:
                    pass
        else:
            print("✅ Reranker already loaded")
        
        self.reranker = ProductMatcherGPU._reranker_instance
    
    def _print_gpu_memory(self):
        """Print GPU memory usage."""
        if self.gpu_available:
            allocated = torch.cuda.memory_allocated(self.gpu_id) / 1e9
            reserved = torch.cuda.memory_reserved(self.gpu_id) / 1e9
            print(f"\n💾 GPU Memory: {allocated:.2f} / {self.vram_total:.1f} GB (reserved: {reserved:.2f} GB)")
    
    def _get_free_vram(self) -> float:
        """Get free VRAM in GB."""
        if not self.gpu_available:
            return 0
        allocated = torch.cuda.memory_allocated(self.gpu_id) / 1e9
        return self.vram_total - allocated
    
    @classmethod
    def clear_models(cls):
        """Clear models and GPU memory."""
        print("🧹 Clearing models...")
        cls._encoder_instance = None
        cls._reranker_instance = None
        gc.collect()
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
            print(f"   GPU memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        
        print("✅ Cleared")
    
    # =========================================================================
    # Text Cleaning
    # =========================================================================
    
    def clean_text(self, text: str) -> str:
        """Text normalization."""
        if pd.isna(text):
            return ""
        
        text = str(text).lower().strip()
        
        # Units normalization (Cyrillic to standard)
        replacements = [
            (r'(\d+)\s*мл\b', r'\1мл'),
            (r'(\d+)\s*ml\b', r'\1мл'),
            (r'(\d+)\s*г\b', r'\1г'),
            (r'(\d+)\s*g\b', r'\1г'),
            (r'(\d+)\s*кг\b', r'\1кг'),
            (r'(\d+)\s*kg\b', r'\1кг'),
            (r'(\d+)\s*л\b', r'\1л'),
            (r'(\d+)\s*l\b', r'\1л'),
            (r'(\d+)\s*шт\b', r'\1шт'),
        ]
        
        for pattern, repl in replacements:
            text = re.sub(pattern, repl, text)
        
        # Cleanup special characters
        text = re.sub(r'[^\w\s\-\.,/]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        
        return text.strip()
    
    def clean_texts_batch(self, texts: List[str], desc: str = "Cleaning") -> List[str]:
        """Batch text cleaning."""
        return [self.clean_text(t) for t in tqdm(texts, desc=desc)]
    
    # =========================================================================
    # Embeddings
    # =========================================================================
    
    def create_embeddings(self, 
                          texts: List[str], 
                          batch_size: int = 8,
                          show_progress: bool = True) -> np.ndarray:
        """Create embeddings (GPU)."""
        
        # Auto-adjust batch_size for RTX 3050
        if self.gpu_available:
            free_vram = self._get_free_vram()
            if free_vram < 1.5:
                batch_size = min(batch_size, 4)
            elif free_vram < 2.0:
                batch_size = min(batch_size, 8)
            elif free_vram < 3.0:
                batch_size = min(batch_size, 16)
            
            print(f"   Batch size: {batch_size} (free VRAM: {free_vram:.1f} GB)")
        
        embeddings = self.encoder.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=show_progress,
            normalize_embeddings=True,
            convert_to_numpy=True,
            device=self.device
        )
        
        return embeddings.astype('float32')
    
    # =========================================================================
    # FAISS
    # =========================================================================
    
    def create_faiss_index(self, embeddings: np.ndarray) -> faiss.Index:
        """Create FAISS index."""
        dimension = embeddings.shape[1]
        
        # CPU index (more reliable)
        index = faiss.IndexFlatIP(dimension)
        index.add(embeddings)
        
        # GPU index if available and memory permits
        if self.faiss_gpu_available and self.gpu_resources and self._get_free_vram() > 0.5:
            try:
                gpu_index = faiss.index_cpu_to_gpu(self.gpu_resources, self.gpu_id, index)
                print(f"   ✅ FAISS on GPU")
                return gpu_index
            except Exception as e:
                print(f"   ⚠️ FAISS on GPU failed: {e}")
        
        print(f"   📊 FAISS on CPU")
        return index
    
    def _index_to_cpu(self, index: faiss.Index) -> faiss.Index:
        """Transfer index to CPU."""
        if hasattr(faiss, 'index_gpu_to_cpu'):
            try:
                return faiss.index_gpu_to_cpu(index)
            except:
                pass
        return index
    
    # =========================================================================
    # Caching
    # =========================================================================
    
    def _get_cache_path(self, name: str) -> Path:
        return self.cache_dir / name
    
    def save_index(self, 
                   df: pd.DataFrame,
                   text_column: str,
                   cache_name: str = 'products'):
        """Create and save index."""
        print("\n" + "=" * 60)
        print(f"💾 Creating Index ({len(df)} items)")
        print("=" * 60)
        
        # Cleaning
        print("\n1️⃣ Cleaning texts...")
        texts_clean = self.clean_texts_batch(df[text_column].tolist())
        
        # Embeddings
        print("\n2️⃣ Creating embeddings...")
        embeddings = self.create_embeddings(texts_clean)
        
        # Index
        print("\n3️⃣ Creating FAISS index...")
        index = self.create_faiss_index(embeddings)
        
        # Saving
        print("\n4️⃣ Saving...")
        cpu_index = self._index_to_cpu(index)
        
        np.save(self._get_cache_path(f'{cache_name}_embeddings.npy'), embeddings)
        pd.DataFrame({'text_clean': texts_clean}).to_parquet(
            self._get_cache_path(f'{cache_name}_texts.parquet')
        )
        faiss.write_index(cpu_index, str(self._get_cache_path(f'{cache_name}_index.faiss')))
        df.to_parquet(self._get_cache_path(f'{cache_name}_products.parquet'))
        
        print(f"✅ Saved to {self.cache_dir}/")
        
        self.faiss_index = index
        self.eva_texts_clean = texts_clean
        self.eva_df = df
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        return index, texts_clean
    
    def load_index(self, cache_name: str = 'products') -> bool:
        """Load index from cache."""
        paths = [
            self._get_cache_path(f'{cache_name}_embeddings.npy'),
            self._get_cache_path(f'{cache_name}_texts.parquet'),
            self._get_cache_path(f'{cache_name}_index.faiss'),
            self._get_cache_path(f'{cache_name}_products.parquet'),
        ]
        
        if not all(p.exists() for p in paths):
            print("⚠️ Cache not found")
            return False
        
        print("\n📂 Loading from cache...")
        
        self.faiss_index = faiss.read_index(str(paths[2]))
        self.eva_texts_clean = pd.read_parquet(paths[1])['text_clean'].tolist()
        self.eva_df = pd.read_parquet(paths[3])
        
        print(f"✅ Loaded {self.faiss_index.ntotal} items")
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        return True
    
    # =========================================================================
    # Search and Reranking
    # =========================================================================
    
    def search_candidates(self, 
                          query_embeddings: np.ndarray,
                          top_k: int = 10) -> Tuple[np.ndarray, np.ndarray]:
        """Search candidates."""
        scores, indices = self.faiss_index.search(query_embeddings.astype('float32'), top_k)
        return indices, scores
    
    def rerank_batch(self,
                     query_texts: List[str],
                     candidates_indices: np.ndarray,
                     batch_size: int = 16) -> List[Tuple[int, float, int]]:
        """Rerank candidates."""
        
        # Collect pairs
        all_pairs = []
        pair_info = []
        
        for q_idx, (query, cand_indices) in enumerate(zip(query_texts, candidates_indices)):
            for pos, idx in enumerate(cand_indices):
                if idx >= 0:
                    all_pairs.append([query, self.eva_texts_clean[idx]])
                    pair_info.append((q_idx, pos, idx))
        
        # Auto-adjust batch_size
        if self.gpu_available:
            free_vram = self._get_free_vram()
            if free_vram < 1.0:
                batch_size = min(batch_size, 8)
            elif free_vram < 1.5:
                batch_size = min(batch_size, 16)
            elif free_vram < 2.0:
                batch_size = min(batch_size, 32)
        
        print(f"   Reranking {len(all_pairs)} pairs (batch={batch_size})...")
        
        # Predict
        all_scores = self.reranker.predict(
            all_pairs, 
            batch_size=batch_size,
            show_progress_bar=True
        )
        
        # Grouping
        query_results = {}
        for (q_idx, pos, idx), score in zip(pair_info, all_scores):
            if q_idx not in query_results:
                query_results[q_idx] = []
            query_results[q_idx].append((idx, float(score), pos))
        
        # Select best for each
        results = []
        for q_idx in range(len(query_texts)):
            if q_idx in query_results:
                best = max(query_results[q_idx], key=lambda x: x[1])
                results.append(best)
            else:
                results.append((-1, 0.0, -1))
        
        return results
    
    # =========================================================================
    # Main Method
    # =========================================================================
    
    def match_products(self,
                       competitor_df: pd.DataFrame,
                       competitor_col: str = 'name',
                       our_df: Optional[pd.DataFrame] = None,
                       our_col: str = 'name',
                       top_k: int = 10,
                       threshold: float = 0.5,
                       encoder_batch: int = 8,
                       reranker_batch: int = 16,
                       use_cache: bool = True,
                       cache_name: str = 'products') -> pd.DataFrame:
        """
        Match products.
        
        Args:
            competitor_df: Competitor DataFrame
            competitor_col: Column with product name
            our_df: Our DataFrame (or None if cached)
            our_col: Column with product name
            top_k: Number of candidates for reranking
            threshold: Confidence threshold
            encoder_batch: Batch size for encoder
            reranker_batch: Batch size for reranker
            use_cache: Use cache
            cache_name: Cache name
        """
        print("\n" + "=" * 70)
        print("🚀 MATCHING PRODUCTS")
        print("=" * 70)
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        # 1. Indexing our products
        if self.faiss_index is None:
            if use_cache and self.load_index(cache_name):
                pass
            elif our_df is not None:
                self.save_index(our_df, our_col, cache_name)
            else:
                raise ValueError("Provide our_df or use cache")
        
        print(f"\n📦 Our products: {self.faiss_index.ntotal}")
        print(f"📦 Competitor products: {len(competitor_df)}")
        
        # 2. Competitor Processing
        print("\n" + "-" * 50)
        print("📝 Stage 1: Preparing competitor products")
        print("-" * 50)
        
        comp_texts = competitor_df[competitor_col].tolist()
        comp_texts_clean = self.clean_texts_batch(comp_texts, "Cleaning")
        
        print("\nCreating embeddings...")
        comp_embeddings = self.create_embeddings(comp_texts_clean, batch_size=encoder_batch)
        
        # 3. Retrieval
        print("\n" + "-" * 50)
        print(f"🔍 Stage 2: Retrieval (Top-{top_k})")
        print("-" * 50)
        
        cand_indices, cand_scores = self.search_candidates(comp_embeddings, top_k)
        print(f"   ✅ Candidates found for {len(cand_indices)} items")
        
        # 4. Reranking
        print("\n" + "-" * 50)
        print("🎯 Stage 3: Reranking")
        print("-" * 50)
        
        rerank_results = self.rerank_batch(comp_texts_clean, cand_indices, batch_size=reranker_batch)
        
        # 5. Results
        print("\n" + "-" * 50)
        print("📊 Stage 4: Building results")
        print("-" * 50)
        
        # Determine column
        if our_col in self.eva_df.columns:
            eva_col = our_col
        else:
            eva_col = self.eva_df.columns[1]
        
        results = []
        for comp_idx, (eva_idx, score, rank) in enumerate(rerank_results):
            
            status = "matched" if score >= threshold else "low_confidence"
            
            results.append({
                'competitor_index': comp_idx,
                'competitor_product': competitor_df.iloc[comp_idx][competitor_col],
                'our_index': eva_idx if status == "matched" else None,
                'our_product': self.eva_df.iloc[eva_idx][eva_col] if eva_idx >= 0 else None,
                'retrieval_score': float(cand_scores[comp_idx][rank]) if rank >= 0 else 0.0,
                'rerank_score': score,
                'retrieval_rank': rank + 1 if rank >= 0 else -1,
                'match_status': status
            })
        
        result_df = pd.DataFrame(results)
        
        # Statistics
        matched = len(result_df[result_df['match_status'] == 'matched'])
        low_conf = len(result_df[result_df['match_status'] == 'low_confidence'])
        
        print(f"\n✅ Done!")
        print(f"   Matched: {matched} ({matched/len(result_df)*100:.1f}%)")
        print(f"   Low confidence: {low_conf} ({low_conf/len(result_df)*100:.1f}%)")
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        return result_df
    
    def analyze_results(self, df: pd.DataFrame, show_examples: int = 5) -> Dict:
        """Analyze results."""
        print("\n" + "=" * 70)
        print("📊 RESULTS ANALYSIS")
        print("=" * 70)
        
        total = len(df)
        matched = df[df['match_status'] == 'matched']
        low_conf = df[df['match_status'] == 'low_confidence']
        
        print(f"\n📈 Statistics:")
        print(f"   Total: {total}")
        print(f"   ✅ Matched: {len(matched)} ({len(matched)/total*100:.1f}%)")
        print(f"   ⚠️  Low confidence: {len(low_conf)} ({len(low_conf)/total*100:.1f}%)")
        
        print(f"\n📉 Rerank score:")
        print(f"   Mean: {df['rerank_score'].mean():.4f}")
        print(f"   Median: {df['rerank_score'].median():.4f}")
        print(f"   Min/Max: {df['rerank_score'].min():.4f} / {df['rerank_score'].max():.4f}")
        
        print(f"\n📊 Distribution:")
        for t in [0.9, 0.7, 0.5, 0.3]:
            count = len(df[df['rerank_score'] >= t])
            print(f"   >= {t}: {count} ({count/total*100:.1f}%)")
        
        if show_examples > 0 and len(matched) > 0:
            print(f"\n📋 Top-{show_examples} Best Matches:")
            print("-" * 70)
            
            for i, (_, row) in enumerate(matched.nlargest(show_examples, 'rerank_score').iterrows(), 1):
                print(f"\n{i}. Score: {row['rerank_score']:.4f}")
                print(f"   Competitor: {row['competitor_product'][:65]}")
                print(f"   Our:        {str(row['our_product'])[:65]}")
        
        if show_examples > 0 and len(low_conf) > 0:
            print(f"\n⚠️  Worst {min(3, len(low_conf))}:")
            print("-" * 70)
            
            for i, (_, row) in enumerate(low_conf.nsmallest(min(3, len(low_conf)), 'rerank_score').iterrows(), 1):
                print(f"\n{i}. Score: {row['rerank_score']:.4f}")
                print(f"   Competitor: {row['competitor_product'][:65]}")
                print(f"   Best Guess: {str(row['our_product'])[:65]}")
        
        return {
            'total': total,
            'matched': len(matched),
            'low_confidence': len(low_conf),
            'avg_score': df['rerank_score'].mean(),
            'median_score': df['rerank_score'].median()
        }


# =============================================================================
# QUICK START
# =============================================================================

def quick_match(competitor_csv: str,
                our_csv: str,
                output_csv: str = 'matches.csv',
                competitor_col: str = 'name',
                our_col: str = 'name',
                threshold: float = 0.5,
                top_k: int = 5) -> pd.DataFrame:
    """
    Quick match from CSV files.
    
    Example:
        results = quick_match('competitor.csv', 'our_products.csv')
    """
    print("📂 Loading data...")
    competitor_df = pd.read_csv(competitor_csv)
    our_df = pd.read_csv(our_csv)
    
    print(f"   Competitor: {len(competitor_df)} items")
    print(f"   Our: {len(our_df)} items")
    
    # Matcher
    matcher = ProductMatcherGPU(use_fp16=True, force_gpu=False)
    
    # Matching (parameters for 4GB VRAM)
    results = matcher.match_products(
        competitor_df,
        competitor_col=competitor_col,
        our_df=our_df,
        our_col=our_col,
        top_k=top_k,
        threshold=threshold,
        encoder_batch=8,
        reranker_batch=16,
        use_cache=True
    )
    
    # Analysis
    matcher.analyze_results(results)
    
    # Save
    results.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"\n💾 Saved: {output_csv}")
    
    return results


if __name__ == "__main__":
    # 1. Check GPU status
    check_gpu_status()

    # 2. Load files
    print("\n📂 LOADING DATA:")
    try:
        # Note: If using semicolon separator, add argument: sep=';'
        df_competitor = pd.read_csv("competitor_products.csv") 
        df_our = pd.read_csv("our_products.csv")
    except FileNotFoundError:
        print("❌ Error: Files not found.")
        print("   Ensure 'competitor_products.csv' and 'our_products.csv' are in the script directory.")
        exit()

    # 3. Column Analysis (Helps determine what to configure)
    print("\n🔍 COLUMN ANALYSIS:")
    print(f"Competitor file (df_competitor): {df_competitor.columns.tolist()}")
    print(f"Our file (df_our):               {df_our.columns.tolist()}")
    print("-" * 60)

    # =========================================================================
    # ⚙️ CONFIGURATION (EDIT THIS BLOCK)
    # =========================================================================
    
    # ❗ Enter exact column names for product names here
    COMPETITOR_COL_NAME = 'name'   # Column name in competitor_products.csv
    OUR_COL_NAME = 'name'          # Column name in our_products.csv

    # Performance Settings
    USE_FP16 = True                # True for RTX 3050 (saves memory)
    BATCH_SIZE_ENCODER = 8         # Use 4 or 8 for 4GB VRAM
    BATCH_SIZE_RERANKER = 16       # Use 16 for 4GB VRAM
    TOP_K_CANDIDATES = 5           # Number of candidates to rerank

    # =========================================================================

    # 4. Initialize Class
    # cache_dir can be specified to avoid re-encoding our products on next run
    matcher = ProductMatcherGPU(
        use_fp16=USE_FP16, 
        cache_dir='./cache_products'
    )
    
    # 5. Start Matching
    try:
        results = matcher.match_products(
            competitor_df=df_competitor,
            competitor_col=COMPETITOR_COL_NAME,
            our_df=df_our,
            our_col=OUR_COL_NAME,
            top_k=TOP_K_CANDIDATES,
            threshold=0.5,             # Confidence threshold (0.0 - 1.0)
            encoder_batch=BATCH_SIZE_ENCODER,
            reranker_batch=BATCH_SIZE_RERANKER,
            use_cache=True             # Use cache for speedup
        )

        # 6. Analyze and Save
        matcher.analyze_results(results, show_examples=5)
        
        output_file = 'results_gpu_matched.csv'
        results.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"\n💾 Results saved to file: {output_file}")

    except KeyError as e:
        print(f"\n❌ COLUMN ERROR: Column not found {e}")
        print("   Check the 'CONFIGURATION' block and compare names with 'COLUMN ANALYSIS' output.")
    except Exception as e:
        print(f"\n❌ An error occurred: {e}")
    finally:
        # Clear GPU memory at the end
        ProductMatcherGPU.clear_models()


🔍 GPU CHECK

📦 PyTorch version: 2.6.0+cu124
🔧 CUDA available: True
🎮 CUDA version: 12.4
📊 GPU count: 1

   GPU 0: NVIDIA GeForce RTX 3050 Laptop GPU
   Memory: 4.3 GB
   Compute capability: 8.6

💾 Current VRAM usage:
   Allocated: 0.00 GB
   Cached: 0.00 GB

📦 FAISS GPU support: False

📂 LOADING DATA:

🔍 COLUMN ANALYSIS:
Competitor file (df_competitor): ['id', 'name']
Our file (df_our):               ['id', 'name']
------------------------------------------------------------
🚀 Initializing ProductMatcherGPU
🖥️  Device: cuda:0
🎮 GPU: NVIDIA GeForce RTX 3050 Laptop GPU
💾 VRAM: 4.3 GB
🔧 FAISS-GPU: ❌ Not installed (Using CPU)
⚡ Precision: FP16

📥 Loading encoder: BAAI/bge-m3


`torch_dtype` is deprecated! Use `dtype` instead!


   ✅ Encoder on: cuda:0

📥 Loading reranker: BAAI/bge-reranker-v2-m3
   ✅ Reranker on: cuda:0

💾 GPU Memory: 2.27 / 4.3 GB (reserved: 2.28 GB)

✅ Initialization complete!

🚀 MATCHING PRODUCTS

💾 GPU Memory: 2.27 / 4.3 GB (reserved: 2.28 GB)
⚠️ Cache not found

💾 Creating Index (500 items)

1️⃣ Cleaning texts...


Cleaning: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 28101.79it/s]


2️⃣ Creating embeddings...
   Batch size: 8 (free VRAM: 2.0 GB)


...


Creating embeddings...
   Batch size: 8 (free VRAM: 2.0 GB)


Batches:   0%|          | 0/63 [00:00<?, ?it/s]


--------------------------------------------------
🔍 Stage 2: Retrieval (Top-5)
--------------------------------------------------
   ✅ Candidates found for 500 items

--------------------------------------------------
🎯 Stage 3: Reranking
--------------------------------------------------
   Reranking 2500 pairs (batch=16)...


Batches:   0%|          | 0/157 [00:00<?, ?it/s]


--------------------------------------------------
📊 Stage 4: Building results
--------------------------------------------------

✅ Done!
   Matched: 500 (100.0%)
   Low confidence: 0 (0.0%)

💾 GPU Memory: 2.28 / 4.3 GB (reserved: 2.33 GB)

📊 RESULTS ANALYSIS

📈 Statistics:
   Total: 500
   ✅ Matched: 500 (100.0%)
   ⚠️  Low confidence: 0 (0.0%)

📉 Rerank score:
   Mean: 0.9911
   Median: 0.9995
   Min/Max: 0.7734 / 1.0000

📊 Distribution:
   >= 0.9: 489 (97.8%)
   >= 0.7: 500 (100.0%)
   >= 0.5: 500 (100.0%)
   >= 0.3: 500 (100.0%)

📋 Top-5 Best Matches:
----------------------------------------------------------------------

1. Score: 1.0000
   Competitor: Pantene ProV Шампунь объем 250мл
   Our:        Pantene Pro-V Shamp объем 250мл

2. Score: 1.0000
   Competitor: L'Oreal Elseve ШАМПУНЬ против выпадения 400мл
   Our:        Shamp Эльсев - против выпадения / 400мл

3. Score: 1.0000
   Competitor: ШАМПУНЬ Elseve для окрашенных волос 250мл
   Our:        для окрашенных волос Засіб д